<a href="https://colab.research.google.com/github/Stp155906/Data-Science-For-Beginners/blob/main/PodaacAPIipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PODAAC CLOUD API S3 Datasets 

In [ ]:
!pip install podaac-data-subscriber

In [ ]:
!git clone -l -s https://github.com/podaac/data-subscriber.git clonedgit

In [ ]:
%cd clonedgit
!ls

In [ ]:
#os.read('podaac_data_subscriber.py')
%cd subscriber/
!ls

In [ ]:
import googleapiclient
#Save first then import 
from google.colab import files
#reading file cloned from github 
files.view('podaac_data_subscriber.py')

open_file = files.view('podaac_data_subscriber.py')



---



In [ ]:
!git clone -l -s https://github.com/podaac/Data-Recipes.git datarecipes


In [ ]:
!pip install prettyprinter

In [ ]:
!pip install utils
#!pip install pprint
!pip install datetime 



In [ ]:
# Accessing Existing collection in PODAAC CLOUD AWS S3

import utils
import pprint
import xarray as xr
from datetime import datetime
import warnings

warnings.simplefilter(action='ignore')

In [ ]:
!git clone https://github.com/podaac/Data-Recipes.git recipes

%cd recipes
!ls


In [ ]:
#ls dataset-introduction 
%cd dataset-introduction/

In [ ]:
ls

In [ ]:
# open file 
import googleapiclient
#Save first then import 
from google.colab import files
#reading file cloned from github 
files.view('utils.py')

open_file = files.view('utils.py')

In [ ]:
# Do not need to run 
files.upload()

In [ ]:
#run(utils.py)

!pip install recipes

In [ ]:
import requests
from pprint import pprint
CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'
collection_url = 'https://cmr.earthdata.nasa.gov/search/collections'
var_url = "https://cmr.earthdata.nasa.gov/search/variables"


# Listing Available datasets

In [ ]:
import utils
import warnings
import pandas as pd
import time

#utils.all_pocloud_dataset()
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 150)
warnings.simplefilter(action='ignore')
data= all_pocloud_dataset('https://archive.podaac.earthdata.nasa.gov/s3credentials')
#data=data.sort_values('time_start')
display(data)
#data.to_html('All_POCLOUD_Data.html')

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

df = pd.DataFrame(data)

df

In [ ]:
#Short_name is used to identify a specific dataset in NASA Earthdata. 
short_name='OISSS_L4_multimission_monthly_v1'

In [ ]:
!pip install requests
!pip install s3fs

In [ ]:
import os 
!pip install --upgrade s3fs
#import s3fs

In [ ]:
!git clone https://github.com/fsspec/s3fs.git 
%cd s3fs
#!python setup.py install


In [ ]:
!pip install awscli

In [ ]:
import awscli
import setuptools


In [ ]:
temp_creds_url = all_pocloud_dataset('https://archive.podaac.earthdata.nasa.gov/s3credentials')



---



In [ ]:
import pprint

#s3://podaac-ops-cumulus-protected is the PODAAC S3 bucket. 
#s3://podaac-ops-cumulus-protected/{short_name} is the path to this dataset. 

#temp_creds_url = all_pocloud_dataset('https://archive.podaac.earthdata.nasa.gov/s3credentials')

!pip install asyncio
!pip install aiobotocore

import asyncio
#from aiobotocore.session import get_session

In [ ]:
!pip install botocore

In [ ]:
!pip install s3fs
#!pip install s3fs --version
!pip install --upgrade s3fs

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
import os
import requests
import botocore
import numpy as np
import xarray as xr
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.plot import show
import rioxarray
#import geoviews as gv

import holoviews as hv
gv.extension('bokeh', 'matplotlib')




---



---



In [ ]:
!pip install panel==0.12.6 hvplot==0.7.3

In [ ]:
import hvplot
import holoviews as hv


In [ ]:
# Create Direct access to s3
def direct_s3(provider='podaac'):
    import requests,s3fs
    s3_cred_endpoint = {
        'podaac':'https://archive.podaac.earthdata.nasa.gov/s3credentials',
        'lpdaac':'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'}

    temp_creds_url = s3_cred_endpoint[provider]
    creds = requests.get(temp_creds_url).json()
    s3 = s3fs.S3FileSystem(anon=False,
                           key=creds['accessKeyId'],
                           secret=creds['secretAccessKey'], 
                           token=creds['sessionToken'])
    return s3

In [ ]:

#For temp access view here
#https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/05_Data_Access_Direct_S3.html

# GHRSST Level 4 AVHRR_OI Global Blended Sea Surface Temperature Analysis (GDS2) from NCEI




---



> https://github.com/podaac/Data-Recipes/blob/main/dataset-introduction/AVHRR_OI-NCEI-L4-GLOB-v2.1.ipynb





---



In [ ]:
!pip install utils


In [ ]:
# Connecting to datasets


import utils
import pprint
import xarray as xr
from datetime import datetime
import warnings

warnings.simplefilter(action='ignore')

In [ ]:
"""
Some ground level functions
"""

import requests
from pprint import pprint
CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'
collection_url = 'https://cmr.earthdata.nasa.gov/search/collections'
var_url = "https://cmr.earthdata.nasa.gov/search/variables"

def find_dataset(provider='podaac',
                 keywords=['swot','level-2']):
    """
    Find a list of collections/datasets that match all the keywords from the keywords list.
    
    
    """
    import pandas as pd

    if 'podaac' in provider.lower().replace('.',''):
        provider='POCLOUD'
        
    response = requests.get(collection_url,params={'cloud_hosted': 'True',
                                        'has_granules': 'True',
                                        'provider': provider,
                                        'page_size':2000,},
                                headers={'Accept': 'application/json', } )
    
    collections = response.json()['feed']['entry']
    
    entries={}
    entries['short_name']=[]
    entries['long_name']=[]
    entries['concept_id']=[]
    entries['time_start']=[]
    entries['time_end']=[]
    
    
    for collection in collections:
        
        title="%s %s %s"%(collection["short_name"],collection["dataset_id"][:97],collection["id"])
        match=1
        for kw in keywords:
            match *= kw.lower() in title.lower()
            
        if match==1:
            entries['short_name'].append(collection["short_name"])
            entries['concept_id'].append(collection["id"])
            entries['long_name'].append(collection["dataset_id"])
            entries['time_start'].append(collection["time_start"])
            try:
                entries['time_end'].append(collection["time_end"])
            except:
                entries['time_end'].append(['NaT/Present'])
    
    return pd.DataFrame(entries)

def all_pocloud_dataset(provider='podaac'):
    """
    a list of all POCLOUD collections.
    """
    import pandas as pd

    if 'podaac' in provider.lower().replace('.',''):
        provider='POCLOUD'
        
    response = requests.get(collection_url,params={'cloud_hosted': 'True',
                                        'provider': provider,
                                        'page_size':2000,},
                                headers={'Accept': 'application/json', } )
    
    collections = response.json()['feed']['entry']
    
    entries={}
    entries['short_name']=[]
    entries['long_name']=[]
    entries['concept_id']=[]
    entries['time_start']=[]
    entries['time_end']=[]
    
    
    for collection in collections:
        
        title="%s %s %s"%(collection["short_name"],collection["dataset_id"][:97],collection["id"])
        match=1
        entries['short_name'].append(collection["short_name"])
        entries['concept_id'].append(collection["id"])
        entries['long_name'].append(collection["dataset_id"])
        entries['time_start'].append(collection["time_start"][:10])
        try:
            entries['time_end'].append(collection["time_end"][:10])
        except:
            entries['time_end'].append(['NaT/Present'])
    
    return pd.DataFrame(entries)

def direct_s3(provider='podaac'):
    import requests,s3fs
    s3_cred_endpoint = {
        'podaac':'https://archive.podaac.earthdata.nasa.gov/s3credentials',
        'lpdaac':'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'}

    temp_creds_url = s3_cred_endpoint[provider]
    creds = requests.get(temp_creds_url).json()
    s3 = s3fs.S3FileSystem(anon=False,
                           key=creds['accessKeyId'],
                           secret=creds['secretAccessKey'], 
                           token=creds['sessionToken'])
    return s3


In [ ]:



def direct_s3(provider='podaac'):
    import requests,s3fs
    s3_cred_endpoint = {
        'podaac':'https://archive.podaac.earthdata.nasa.gov/s3credentials',
        'lpdaac':'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'}

    temp_creds_url = s3_cred_endpoint[provider]
    creds = requests.get(temp_creds_url).json()
    s3 = s3fs.S3FileSystem(anon=False,
                           key=creds['accessKeyId'],
                           secret=creds['secretAccessKey'], 
                           token=creds['sessionToken'])
    return s3


In [ ]:
import collections

archive_files = 'https://archive.podaac.earthdata.nasa.gov/s3credentials'
other_files = 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'

CMR_OPS
direct_s3
#podaac_s3='podaac-ops-cumulus-protected'

#395 total collections in PODAAC CLOUD (POCLOUD) by 03/25/2022.
print(len(temp_creds_url), "total collections in PODAAC CLOUD (POCLOUD) by %s."%datetime.today().strftime('%m/%d/%Y'))



In [ ]:
find_dataset(keywords=['AVHRR'])

wind_data = find_dataset(keywords=['AVHRR'])

display(wind_data)




In [ ]:
#find_dataset(keywords=['Sea Surface Temperature'])


Sea_Surface_Temperature = find_dataset(keywords=['Sea Surface Temperature'])

display(Sea_Surface_Temperature)

In [ ]:
!pip install datetime

In [ ]:


from datetime import time

Sea_Surface_Temperature = find_dataset(keywords=['1992'])

display(Sea_Surface_Temperature)

In [ ]:

import datetime as dt
# or
from datetime import date

d1 = dt.date(2020, 11, 19)
d1
#datetime.date(2020, 11, 19)




In [ ]:
 d1.year

In [ ]:
d1.month

In [ ]:
d1.day

In [ ]:
oldest = Sea_Surface_Temperature['time_start'].min() # Earliest date
newest = Sea_Surface_Temperature['time_end'].max() # Latest date


print(newest)
print(oldest)


In [ ]:
!pip install numpy
import numpy as np


In [ ]:


# Dropna

df= Sea_Surface_Temperature.dropna(how= 'any', axis =0).reset_index()


missing_vals = ["NA","", "[NaT/Present]",None, np.NAN]
missing = df.isin(missing_vals)
missing.head()

df.fillna(0).head()


In [ ]:

oldest = df['time_start'].min() # Earliest date
#newest = df['time_end'].max() # Latest date


#print(newest)
print(oldest)

#Detect a list of missing values with df.isin



In [ ]:
#!pip install DataFrame
from pandas import DataFrame

oldest = df['time_start'][38]

df



---



---



[Time Series Panda resource](https://pandas.pydata.org/docs/user_guide/timeseries.html)

In [ ]:
short_name = 'REYNOLDS_NCDC_L4_MONTHLY_V5'

In [ ]:
import pprint

In [ ]:
from typing_extensions import get_origin
origin_value = get_origin(short_name)




In [ ]:
#short_name="AVHRR_OI-NCEI-L4-GLOB-v2.1"
fns=sorted(podaac_s3+'/%s/*nc'%short_name)
print('There are %i files in this dataset.'%len(fns))
print('The first five files are:')
pprint.pprint(fns[:])
print('The last five files are:')
pprint.pprint(fns[-5:])

In [ ]:
!pip install podaac_data_subscriber
#!pip install podaac

In [ ]:
def test_downloader_GRACE_with_SHA_512(tmpdir):
    # start with empty directory
    directory_str = str(tmpdir)
    assert len( os.listdir(directory_str) ) == 0

    # run the command once -> should download the file. Note the modified time for the file
    args = create_downloader_args(f"-c GRACEFO_L2_CSR_MONTHLY_0060 -sd 2020-01-01T00:00:00Z -ed 2020-01-02T00:00:01Z -d {str(tmpdir)} --limit 1 --verbose -e 00".split())
    pdd.run(args)
    assert len( os.listdir(directory_str) ) > 0
    filename = directory_str + "/" + os.listdir(directory_str)[0]
    modified_time_1 = os.path.getmtime(filename)
    print( modified_time_1 )

In [ ]:
import pytest
import os
from os.path import exists
from subscriber import podaac_data_downloader as pdd
import shutil
from pathlib import Path


In [ ]:
# Getting a url from this dataset

!pip install urlextract

# A simple regex for URL matching like the following should fit your case.
from urllib.parse import urlparse

In [ ]:
!pip install wget
import wget

In [ ]:
from sys import meta_path
from numpy.ma.core import get_data
from traitlets.traitlets import List
from posix import listdir

#from traitlets.traitlets import List
from urllib.request import url2pathname
#get_origin(Literal[38])


df.


In [ ]:
from urlextract import URLExtract

extractor = URLExtract()
urls = extractor.find_urls('NOAA Smith and Reynolds Extended Reconstructed Sea Surface Temperature (ERSST) Level 4 Monthly Version 5 Dataset in netCDF')
print(urls) # prints: ['stackoverflow.com']



---



In [ ]:
#find_dataset(keywords=['NOAA Smith and Reynolds Extended Reconstructed Sea Surface Temperature (ERSST) Level 4 Monthly Version 5 Dataset in netCDF'])

#wind_data = find_dataset(keywords=['NOAA Smith and Reynolds Extended Reconstructed Sea Surface Temperature (ERSST) Level 4 Monthly Version 5 Dataset in netCDF'])

df[:]

find_dataset()
#source_type= findsource(keywords= ['REYNOLDS_NCDC_L4_MONTHLY_V5 '])
#display(source_type)


In [ ]:
!wget < https://archive.podaac.earthdata.nasa.gov/s3credentials>